In [ ]:
# Base Worker for Megadrive transfer

from google.colab import output
import logging, threading, time, random


class Megadrive(BaseWorker):
    def __init__(self, name, state):
        # Don't overwrite this method. Use self.set() instead
        super().__init__(name, state)
        self.files = ['list.txt']
        self.file_completed = 0
        self.file_onqueue = []
        self.file_total = 0
        self.file_working = 0
        self.file_current = ''
        self.file_rawlist = []
        # Set function
        self.set()

    def working(self):
        # Initiation
        print('----- Initiation -----')
        logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')
        if not self.init():
            self.success = False
            return
        output.clear()
        print('Initiation Done ...')
        # Read files
        print('----- Reading files -----')
        self.read_phase()
        # Check files
        print('----- Checking files -----')
        self.check_phase()
        # Transfer files
        print('----- Transferring files -----')
        self.transfer_phase()
        # Done
        print('----- DONE -----')

    def get_result_path(self, raw_path, from_path, to_path):
        parent_path, file_name = os.path.split(raw_path)
        if parent_path.startswith(from_path):
            parent_path = parent_path[len(from_path):]
        result_folder = os.path.join(to_path, parent_path.lstrip('/'))
        result_file = os.path.join(result_folder, file_name)
        return result_folder, result_file

    # Megadrive Phases (can be implemented)
    def read_phase(self):
        # Read file from list.txt
        self.file_rawlist = self.read_file('list.txt', read_empty=False)

    def check_phase(self):
        # Receive file list from self.file_rawlist and return self.file_completed, self.file_onqueue, self.file_total
        for raw in self.file_rawlist:
            if raw:
                if self.check(raw):
                    self.file_completed += 1
                else:
                    self.file_onqueue.append(raw)
        self.file_total = len(self.file_onqueue)
        print(f'Checking done. {self.file_completed} files completed. {self.file_total} files on queue.')
        if self.file_total > 0:
            self.log('INFO', f'{self.file_completed} files completed. {self.file_total} files on queue.')
            self.endmsg = f'Completed. {self.file_total} transfered. {self.file_completed} files completed'
        else:
            self.endmsg = f'Nothing to transfer. {self.file_completed} files completed.'
            return

    def transfer_phase(self):
        # Transfer files from file list in self.file_onqueue. Update self.file_working and self.file_current
        threading.Thread(target=self.biglog).start()
        for file_path in self.file_onqueue:
            self.file_working += 1
            self.file_current = file_path
            self.smalllog()
            self.transfer(file_path)

    # Implementation methods

    def set(self):
        # Implement this. Use to declare class variables
        # This will be called before Prepare phase
        pass

    def init(self):
        # Implement this. Return True when initiation is completed, return False otherwise
        # This will be called after Prepare phase
        pass
    
    def check(self, path):
        # Implement this. Return True when the file is already existed, return False otherwise
        pass
    
    def transfer(self, path):
        # Implement this
        pass

    def smalllog(self):
        # Implement this
        msg = 'Transfering: ' + str(self.file_working) + ' / ' + str(self.file_total) + '\nCompleted: ' + str(self.file_completed) + '\n' + self.file_current
        print()
        logging.info(msg)
    
    def biglog(self):
        # Implement this
        while True:
            time.sleep(3600 + random.randint(0, 600))
            msg = 'Transfering: ' + str(self.file_working) + ' / ' + str(self.file_total) + ' . Completed: ' + str(self.file_completed)
            self.log('INFO', msg)
